### steps
1. Create an sample SQL database and table for testing cases
2. performing SQL window on database to solve Consecute number problem

In [45]:
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object

In [44]:
def create_msql_database(name_database):
    """Function create MYSQL database"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    databases = [x[0] for x in mycursor]
    if name_database not in databases:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")
    # Check database


In [17]:
def create_msql_table(name_table):
    """Connect to MYSQL database"""

    db_config = read_db_config()
    conn = None
    try:
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)

        if conn.is_connected():
            print('Connection established')
        else:
            print('Connection failed')

    except Error as e:
        print(e)



1049 (42000): Unknown database 'python_my'
